In [3]:
!pip install weaviate-client
!pip install datasets

/Users/jb/miniconda3/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=5875) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


In [43]:
# declare name of the collection
COLLECTION_NAME = "QuestionAnswering"
QUERY_SENTENCE = "사이보그가 뭐야?"
NUM_OF_DATA = 200
WEIGHT_OF_VECTOR_IN_HYBRID = 0.5

In [46]:
import pandas as pd
from datasets import load_dataset

DATA_SET    = "beomi/KoAlpaca-v1.1a"
DATA_FILE   = "./data/KoAlpaca-train.csv"

load_data = load_dataset(DATA_SET, split="train")
load_data.to_csv(DATA_FILE)
csv_data = pd.read_csv(DATA_FILE)
csv_data.head(5)
data_to_insert = csv_data.head(min(NUM_OF_DATA, len(load_data)))

Creating CSV from Arrow format:   0%|          | 0/22 [00:00<?, ?ba/s]

In [42]:
import weaviate

client = weaviate.connect_to_local(host='localhost', port=8080)

In [44]:
from weaviate.collections.classes.config import Tokenization
import weaviate.classes.config as wc

# Drop the collection
client.collections.delete(name=COLLECTION_NAME)

client.collections.create(
    name=COLLECTION_NAME,
    description=COLLECTION_NAME,
    properties=[
        wc.Property(
            name="instruction",
            data_type=wc.DataType.TEXT,
            tokenization=Tokenization.KAGOME_KR
        ),
        wc.Property(
            name="output",
            data_type=wc.DataType.TEXT,
            tokenization=Tokenization.KAGOME_KR
        ),
        wc.Property(
            name="url",
            data_type=wc.DataType.TEXT,
            skip_vectorization=True),
    ],
    vectorizer_config=wc.Configure.Vectorizer.text2vec_transformers(),
)

In [47]:
from weaviate.collections.classes.filters import Filter

target_collection = client.collections.get(COLLECTION_NAME)

# Fetch the first 200 rows from csv_data
data_to_insert = csv_data.head(min(NUM_OF_DATA, len(csv_data)))

with target_collection.batch.dynamic() as batch:
    for _, row in data_to_insert.iterrows():
        # 중복 확인
        response = target_collection.query.fetch_objects(
            filters=Filter.by_property("instruction").equal(row["instruction"]),
            limit=1
        )
        # object가 있으면 건너뛰기
        if response.objects:
            continue

        batch.add_object(
            properties={
                "instruction": row["instruction"],
                "output": row["output"],
                "url": row["url"],
            },
        )


In [48]:
from weaviate.classes.query import MetadataQuery

target_collection = client.collections.get(COLLECTION_NAME)

response = target_collection.query.near_text(
    query=QUERY_SENTENCE,
    limit=3,
    return_metadata=MetadataQuery(distance=True)
)

for o in response.objects:
    print(o.properties)
    print(o.metadata.distance)

{'instruction': '"데요"와 "대요"의 차이점은 무엇인가요? "이상한데?"나 "아닌데?" 같은 경우도 포함되나요? 뉴스를 보고 나서 "내일 비 온다"라면 "내일 비 온대"와 "내일 비 온데"라는 표현의 차이점은 무엇인가요? "어째 이상하데?" 같은 경우에는 어떤 상황에서 사용되나요?', 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=11080103&docId=275232662', 'output': '"-대"를 "-다고 해"를 줄인 것이라는 것을 이해하는 것이 중요합니다. "-다고 해"와 "-대"는 어떤 것을 표현할 때 사용하는 것이기 때문에, "-대"는 자신이 본 것을 말할 때, "-다"는 타인의 언급을 전달할 때 주로 사용됩니다. 따라서 "어째 이상하데?"와 같이 직접적인 언급을 할 때도 "-데"를 사용합니다. \n\n그리고 "이상한데?"나 "아닌데?"와 같은 경우, "이상하다"나 "아니다"와 같은 형용사가 바로 앞에 나와 있기 때문에 "-ㄴ데"라는 어미를 사용합니다. 또한 뉴스를 보고 "내일 비 온다"와 같이 언급한 경우, 이는 기상캐스터나 뉴스에서 발표한 내용을 전달하는 것이기 때문에 "-대"를 사용합니다. \n\n예를 들어 "뉴스에서 내일 비 온대. 우산 챙기세요."라면, 누군가가 내일 비가 올 것이라고 전하는 것이기 때문에 "-대"를 사용하며, "내일 비 온데. 우산 챙기세요."라면, 직접 기상 관측을 한 것이 아니고, 뉴스를 보고 비가 올 것이라는 것을 추측한 것이기 때문에 "-데"를 사용합니다. \n\n마지막으로 "어째 이상하데?" 같은 경우에는, \'어째\', \'어쩌다\', \'왜\'와 같은 의문어와 같이 이상한 상황이나 모순되는 상황을 표현할 때 사용합니다. \n\n따라서, "-ㄴ데"는 형용사나 형태소 앞에서, "-대"는 타인이 언급한 것을 전달하거나 추측할 때, "-데"는 직접 관찰한 것이 아닌 추측이나 전해 들은 것을 전달할 때, "어째 이상하데?"와 같은 경우는 

In [49]:
target_collection = client.collections.get(COLLECTION_NAME)

response = target_collection.query.bm25(
    query=QUERY_SENTENCE,
    return_metadata=MetadataQuery(score=True),
    limit=3
)

for o in response.objects:
    print(o.properties)
    print(o.metadata.score)

{'instruction': "'사이보그'는 언제 처음 등장한 말이며, 그 의미와 종류에는 어떤 것이 있는지 알고 싶습니다.", 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=110801&docId=53501505', 'output': "'사이보그'는 1960년에 처음 등장한 말로, 기계와 유기체가 합성되어 생겨난 새로운 개념을 나타냅니다. 주로 공상과학영화에서 등장하며, 1985년 페미니스트인 도나 해러웨이가 발표한 '사이보그를 위한 선언문'으로 인해 사이보그는 성차별 사회를 극복하는 사회정치적 상징으로 더욱 주목받게 되었습니다. 사이보그는 생물체와 기술적 개입이 결합된 다양한 종류가 있으며, 이제는 인공장기를 받은 사람, 예방접종을 받은 사람, 또는 사람과 기계가 결합된 베리칩과 같은 기술적 개입을 받은 사람들까지 우리 주변에서 만날 수 있습니다. \n\n또한, 지구 위치 파악 기능이나 신체 건강 상태를 감지하는 센서 등을 추가적으로 부착하면서 사이보그의 구조와 기능이 더욱 다양해지고 있습니다. 이러한 발전은 사이보그의 대중화를 가속화시키고 있으나, 범죄나 인권 침해 등의 문제가 발생할 가능성도 존재합니다."}
3.397840738296509
{'instruction': '야구 글러브의 종류와 포지션별 특징은 무엇인가요?', 'url': 'https://kin.naver.com/qna/detail.naver?d1id=10&dirId=100102&docId=282734372', 'output': '보통 4칸으로 그물처럼 되어있는 것이 내야글러브이며, 1루를 제외한 내야 전 포지션에서 사용 가능합니다. 그물처럼 촘촘히 되어있는 것이 외야글러브이며, 우익수, 중견수, 좌익수에서 사용 가능합니다. 투수 글러브는 그립이 보이지 않도록 막혀 있습니다. 포수와 1루수는 글러브가 아닌 미트를 사용합니다.'}
2.775923490524292
{'output': '엑스레이 사진은 전자를 물체에 쏘아 물질을 투과한

In [37]:
target_collection = client.collections.get(COLLECTION_NAME)

response = target_collection.query.hybrid(
    query=QUERY_SENTENCE,
    alpha=WEIGHT_OF_VECTOR_IN_HYBRID,
    return_metadata=MetadataQuery(score=True, explain_score=True),
    limit=3,
)

for o in response.objects:
    print(o.properties)
    print(o.metadata.score, o.metadata.explain_score)

{'input': "격조사 '가'가 붙는 경우 '네가'를 사용해야 합니다. '니'는 표준어가 아닌 언어 사용이며 격조사 '가'가 붙는 경우에 '네가' 대신 '니가'를 사용하는 것은 옳지 않습니다. 따라서 제시된 질문 '니가랑 네가중에 뭐가 맞는 말인가요?'는 '네가랑 네가중에 뭐가 맞는 말인가요?'로 수정해야합니다. \n 또한, '너'는 '네'로 바뀌며 '니'라는 표준어가 아닌 사용은 지양하는 것이 바람직합니다.", 'instruction': "'니가랑 네가중에 뭐가 맞는 말인가요?' 라는 질문의 올바른 표현은 무엇인가요? 그리고 그 이유는 무엇인가요?", 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=11080103&docId=387957459'}
0.5256288051605225 
Hybrid (Result Set keyword,bm25) Document 5a4b97f7-5958-474b-96f7-7d34b585f309: original score 2.976395, normalized score: 0.23197371 - 
Hybrid (Result Set vector,hybridVector) Document 5a4b97f7-5958-474b-96f7-7d34b585f309: original score 0.6007407, normalized score: 0.29365513
{'input': "'사이보그'는 1960년에 처음 등장한 말로, 기계와 유기체가 합성되어 생겨난 새로운 개념을 나타냅니다. 주로 공상과학영화에서 등장하며, 1985년 페미니스트인 도나 해러웨이가 발표한 '사이보그를 위한 선언문'으로 인해 사이보그는 성차별 사회를 극복하는 사회정치적 상징으로 더욱 주목받게 되었습니다. 사이보그는 생물체와 기술적 개입이 결합된 다양한 종류가 있으며, 이제는 인공장기를 받은 사람, 예방접종을 받은 사람, 또는 사람과 기계가 결합된 베리칩과 같은 기술적 개입을 받은 사람들까지 우리 주변에서 만날 수 